In [307]:
from argparse import ArgumentParser
from google.cloud import bigquery
import pandas as pd
import datetime

In [308]:
this_year = datetime.datetime.now().year
last_year =(this_year-1)
form_to_check = "RR-20"
worksheet= "Financials - 2"


In [83]:
bq_form_ref = form_to_check.replace("-","").lower() #this is something like "rr20" for the "RR-20" form_to_check
bq_sheet_ref = worksheet.replace(" ", "_").replace("/", "_").replace(".", "_").replace("-", "").replace('\W+', '').lower()


### How to check, for which orgs to run these checks? 
1. We want all orgs that do not have that check yet. 
  * can make a bookkeeping table that records Y/N for when each org runs each kind of check
2. We want all orgs that submitted a revision.


#### FOR NOW:
For each org, filter to the rows with the latest `date_uploaded`. 
* Each revision that an agency submits into BlackCat should be in the `blackcat_raw` data table in BigQuery. By grabbing all rows with the latest date for each agency, we get the latest submitted report info.
* Since 2023 and 2022 data is kept in different tables, do this for each table and we will have the latest data submittal for each year.


#### FOR ETL: doing week of 10/16/23
Make bookkeeping table of which checks have been completed by which orgs, check against said table, and only pull the data for the orgs that still need to have these validation checks run.

In [260]:
### Query "For Now"
bq_data_query = f"""SELECT * FROM 
      (select *,
      RANK() OVER(PARTITION BY Organization_Legal_Name ORDER BY date_uploaded DESC) rank_date 
    from `cal-itp-data-infra.blackcat_raw.{this_year}_{bq_form_ref}_{bq_sheet_ref}`) s 
    WHERE rank_date = 1;
    """

print(bq_data_query)


SELECT * FROM 
      (select *,
      RANK() OVER(PARTITION BY Organization_Legal_Name ORDER BY date_uploaded DESC) rank_date 
    from `cal-itp-data-infra.blackcat_raw.2023_rr20_financials__2`) s 
    WHERE rank_date = 1;
    


In [251]:
client = bigquery.Client()

## we don't really need this
# results = bq_data_query.result() # Wait for the job to complete.
# results = client.query(bq_data_query).result()
# results

In [262]:
rr20_financial = client.query(bq_data_query).to_dataframe()
rr20_financial = rr20_financial.drop_duplicates().drop('rank_date', axis=1)
rr20_financial

,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Operating_Capital,Total_Annual_Revenues_Expended,Total_Annual_Expenses_by_Mode,Fare_Revenues,Other_Directly_Generated_Funds,Revenues_Accrued_Through_a_PT_Agreement,NonFederal_Funds,FTA_Metropolitan_Planning_5303,FTA_Urbanized_Area_Formula_Program_5307,FTA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,ARRA_Urbanized_Area_Program_Funds_5307,ARRA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,CARES_Act_Urbanized_Area_Program_Funds_5307,CRRSA_Act_Urbanized_Area_Program_Funds_5307,American_Rescue_Plan_Act_of_2021_Urbanized_Area_Program_Funds_5307,FTA_Clean_Fuels_Program_5308,FTA_Capital_Investment_Grants_5309,ARRA_Major_Capital_Investment_New_Starts_Funds_5309,American_Rescue_Plan_Act_of_2021_Fixed_Guideway_Capital_Investment_Grants_5303,FTA_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Formula_Program_5310,Capital_Assistance_Spent_on_Operations_5310,CRRSA_Act_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,American_Rescue_Plan_Act_of_2021_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,FTA_Formula_Grants_for_Rural_Areas_5311,Capital_Assistance_Spent_on_Operations_5311,FTA_ARRA_Other_than_Urbanized_Area_Program_Funds_5311,FTA_ARRA_Capital_Assistance_Spent_on_Operations_including_maintenance_expenses_5311,FTA_Tribal_Transit_Funds_5321,ARRA_Tribal_Transit_Funds_5311,CARES_Act_Rural_Area_Program_Funds_5311,CARES_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5311,CRRSA_Act_Rural_Area_Program_Funds_5311,CRRSA_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,American_Rescue_Plan_Act_of_2021_Rural_Area_Program_Funds_5311,American_Rescue_Plan_Act_of_2021_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,FTA_Job_Access_and_Reverse_Commute_Formula_Program_5316,State_of_Good_Repair_5308,FTA_Bus_and_Bus_Facilities,ARRA_TIGGER_Greenhouse_Gas_and_Energy_Reduction,Other_FTA_Funds,Other_USDOT_Funds,Other_Federal_Funds,date_uploaded
0,County of Sacramento Department of Transportation,None,2023,Capital,0,0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-10-11
1,County of Sacramento Department of Transportation,None,2023,Operating,2152007,2152007,108270.0,NaN,NaN,1302377.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,420860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,320500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-10-11
2,Amador Transit,None,2023,Capital,0,0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2023-10-11
3,Amador Transit,None,2023,Operating,1484540,1484440,82216.0,5282.0,NaN,671952.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,56484.0,239425,NaN,NaN,NaN,NaN,NaN,226345.0,NaN,202836.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2023-10-11
4,San Benito County Local Transportation Authority,None,2023,Capital,180873,180873,0.0,0.0,NaN,180873.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-10-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Mendocino Transit Authority,MTA,2023,Operating,6284790,6284790,254526.0,389565.0,0.0,1629772.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54665.0,NaN,NaN,NaN,717774,NaN,NaN,NaN,NaN,NaN,1287311.0,NaN,1459704.0,NaN,491473.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-10-11
72,Eureka Transit Service,None,2023,Capital,0,0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,2023-10-11
73,Eureka Trans

In [261]:
bq_2022_query = f"""SELECT * FROM `cal-itp-data-infra.blackcat_raw.{last_year}_{bq_form_ref}_{bq_sheet_ref}` ;
    """
print(bq_2022_query)
rr20_financial_2022 = client.query(bq_2022_query).to_dataframe()
rr20_financial_2022 = rr20_financial_2022.drop_duplicates()
rr20_financial_2022

SELECT * FROM `cal-itp-data-infra.blackcat_raw.2022_rr20_financials__2` ;
    


,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Operating_Capital,Total_Annual_Revenues_Expended,Total_Annual_Expenses_by_Mode,Fare_Revenues,Other_Directly_Generated_Funds,Revenues_Accrued_Through_a_PT_Agreement,NonFederal_Funds,FTA_Metropolitan_Planning_5303,FTA_Urbanized_Area_Formula_Program_5307,FTA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,ARRA_Urbanized_Area_Program_Funds_5307,CARES_Act_Urbanized_Area_Program_Funds_5307,CRRSA_Act_Urbanized_Area_Program_Funds_5307,American_Rescue_Plan_Act_of_2021_Urbanized_Area_Program_Funds_5307,FTA_Clean_Fuels_Program_5308,FTA_Capital_Investment_Grants_5309,ARRA_Major_Capital_Investment_New_Starts_Funds_5309,American_Rescue_Plan_Act_of_2021_Fixed_Guideway_Capital_Investment_Grants_5303,FTA_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Formula_Program_5310,Capital_Assistance_Spent_on_Operations_5310,CRRSA_Act_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,American_Rescue_Plan_Act_of_2021_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,FTA_Formula_Grants_for_Rural_Areas_5311,Capital_Assistance_Spent_on_Operations_5311,FTA_ARRA_Other_than_Urbanized_Area_Program_Funds_5311,FTA_ARRA_Capital_Assistance_Spent_on_Operations_including_maintenance_expenses_5311,FTA_Tribal_Transit_Funds_5321,ARRA_Tribal_Transit_Funds_5311,CARES_Act_Rural_Area_Program_Funds_5311,CARES_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5311,CRRSA_Act_Rural_Area_Program_Funds_5311,CRRSA_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,American_Rescue_Plan_Act_of_2021_Rural_Area_Program_Funds_5311,American_Rescue_Plan_Act_of_2021_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,FTA_Job_Access_and_Reverse_Commute_Formula_Program_5316,State_of_Good_Repair_5308,FTA_Bus_and_Bus_Facilities,ARRA_TIGGER_Greenhouse_Gas_and_Energy_Reduction,Other_FTA_Funds,Other_USDOT_Funds,Other_Federal_Funds
0,City of Ojai,None,2022,Operating,733466,733466,39335.0,40460.0,NaN,465385.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185951.0,NaN,NaN,NaN,NaN,NaN,2335.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,City of Ojai,None,2022,Capital,0,0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,City of Taft,None,2022,Capital,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,City of Taft,None,2022,Operating,394016,394016,55871.0,12.0,0.0,305234.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32899.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,City of Arvin,None,2022,Operating,853504,853504,36386.0,NaN,NaN,548259.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73230.0,NaN,NaN,NaN,NaN,NaN,195629.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Tehama County Transit Agency,TRAX,2022,Capital,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
92,Tehama County Transit Agency,TRAX,2022,Operating,2029474,2029474,0.0,0.0,0.0,914771.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150000.0,NaN,37674.0,0.0,398903.0,0.0,NaN,0.0,NaN,0.0,528126.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
93,Tuolumne County Transit Agency (TCTA),TCT,2022,Capital,214680,214680,0.0,0.0,NaN,214680.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,Tuolumne County Transit Agency (TCTA),TCT,2022,Operating,2393800,2393800,4617.0,44338.0,NaN,1267532.0,N

Checks created on Excel files 

In [84]:
# rr20_financial = pd.read_excel("../data/NTD_Annual_Report_Rural_2023_2023-09-28.xlsx", 
#                      sheet_name="Financials - 2", index_col=None) 
# rr20_financial_2022 = pd.read_excel("../data/NTD_Annual_Report_Rural_2022.xlsx", 
                     sheet_name="Financials - 2", index_col=None) 
orgs = pd.read_csv("../data/organizations.csv")

In [86]:
### This was for the excel sheet import - 
## Remove spaces and slashes from col names - this is what they will be called in BQ
# rr20_financial.columns = (rr20_financial.columns.str.replace(' ', '_')
#                             .str.replace('/', '_').str.replace('.', '_')
#                              .str.replace('-', '')
#                              .str.replace('#', 'num')
#                             .str.replace('\W+', '') #other things, just strip out
#                             )
# rr20_financial.head(3)

/var/folders/3h/9n_2y6x178l41fsp4dg802f00000gn/T/ipykernel_12588/3231551084.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rr20_financial.columns = (rr20_financial.columns.str.replace(' ', '_')
/var/folders/3h/9n_2y6x178l41fsp4dg802f00000gn/T/ipykernel_12588/3231551084.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  rr20_financial.columns = (rr20_financial.columns.str.replace(' ', '_')


,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Operating_Capital,Total_Annual_Revenues_Expended,Total_Annual_Expenses_by_Mode,Fare_Revenues,Other_Directly_Generated_Funds,Revenues_Accrued_Through_a_PT_Agreement,NonFederal_Funds,FTA_Metropolitan_Planning_5303,FTA_Urbanized_Area_Formula_Program_5307,FTA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,ARRA_Urbanized_Area_Program_Funds_5307,ARRA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,CARES_Act_Urbanized_Area_Program_Funds_5307,CRRSA_Act_Urbanized_Area_Program_Funds_5307,American_Rescue_Plan_Act_of_2021_Urbanized_Area_Program_Funds_5307,FTA_Clean_Fuels_Program_5308,FTA_Capital_Investment_Grants_5309,ARRA_Major_Capital_Investment_New_Starts_Funds_5309,American_Rescue_Plan_Act_of_2021_Fixed_Guideway_Capital_Investment_Grants_5303,FTA_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Formula_Program_5310,Capital_Assistance_Spent_on_Operations_5310,CRRSA_Act_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,American_Rescue_Plan_Act_of_2021_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,FTA_Formula_Grants_for_Rural_Areas_5311,Capital_Assistance_Spent_on_Operations_5311,FTA_ARRA_Other_than_Urbanized_Area_Program_Funds_5311,FTA_ARRA_Capital_Assistance_Spent_on_Operations_including_maintenance_expenses_5311,FTA_Tribal_Transit_Funds_5321,ARRA_Tribal_Transit_Funds_5311,CARES_Act_Rural_Area_Program_Funds_5311,CARES_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5311,CRRSA_Act_Rural_Area_Program_Funds_5311,CRRSA_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,American_Rescue_Plan_Act_of_2021_Rural_Area_Program_Funds_5311,American_Rescue_Plan_Act_of_2021_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,FTA_Job_Access_and_Reverse_Commute_Formula_Program_5316,State_of_Good_Repair_5308,FTA_Bus_and_Bus_Facilities,ARRA_TIGGER_Greenhouse_Gas_and_Energy_Reduction,Other_FTA_Funds,Other_USDOT_Funds,Other_Federal_Funds
0,City of Arvin,NaN,2023,Capital,0,0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,City of Arvin,NaN,2023,Operating,982115,963938,42028.0,18177.0,NaN,845876.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,City of Corcoran - Corcoran Area Transit,NaN,2023,Capital,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
# rr20_financial_2022.columns = (rr20_financial_2022.columns.str.replace(' ', '_')
#                             .str.replace('/', '_').str.replace('.', '_')
#                              .str.replace('-', '')
#                              .str.replace('#', 'num')
#                             .str.replace('\W+', '') #other things, just strip out
#                             )
# print(rr20_financial_2022.shape)
# rr20_financial_2022.head(3)

(98, 45)


/var/folders/3h/9n_2y6x178l41fsp4dg802f00000gn/T/ipykernel_12588/2275517188.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rr20_financial_2022.columns = (rr20_financial_2022.columns.str.replace(' ', '_')
/var/folders/3h/9n_2y6x178l41fsp4dg802f00000gn/T/ipykernel_12588/2275517188.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  rr20_financial_2022.columns = (rr20_financial_2022.columns.str.replace(' ', '_')


,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Operating_Capital,Total_Annual_Revenues_Expended,Total_Annual_Expenses_by_Mode,Fare_Revenues,Other_Directly_Generated_Funds,Revenues_Accrued_Through_a_PT_Agreement,NonFederal_Funds,FTA_Metropolitan_Planning_5303,FTA_Urbanized_Area_Formula_Program_5307,FTA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,ARRA_Urbanized_Area_Program_Funds_5307,ARRA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,CARES_Act_Urbanized_Area_Program_Funds_5307,CRRSA_Act_Urbanized_Area_Program_Funds_5307,American_Rescue_Plan_Act_of_2021_Urbanized_Area_Program_Funds_5307,FTA_Clean_Fuels_Program_5308,FTA_Capital_Investment_Grants_5309,ARRA_Major_Capital_Investment_New_Starts_Funds_5309,American_Rescue_Plan_Act_of_2021_Fixed_Guideway_Capital_Investment_Grants_5303,FTA_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Formula_Program_5310,Capital_Assistance_Spent_on_Operations_5310,CRRSA_Act_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,American_Rescue_Plan_Act_of_2021_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,FTA_Formula_Grants_for_Rural_Areas_5311,Capital_Assistance_Spent_on_Operations_5311,FTA_ARRA_Other_than_Urbanized_Area_Program_Funds_5311,FTA_ARRA_Capital_Assistance_Spent_on_Operations_including_maintenance_expenses_5311,FTA_Tribal_Transit_Funds_5321,ARRA_Tribal_Transit_Funds_5311,CARES_Act_Rural_Area_Program_Funds_5311,CARES_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5311,CRRSA_Act_Rural_Area_Program_Funds_5311,CRRSA_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,American_Rescue_Plan_Act_of_2021_Rural_Area_Program_Funds_5311,American_Rescue_Plan_Act_of_2021_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,FTA_Job_Access_and_Reverse_Commute_Formula_Program_5316,State_of_Good_Repair_5308,FTA_Bus_and_Bus_Facilities,ARRA_TIGGER_Greenhouse_Gas_and_Energy_Reduction,Other_FTA_Funds,Other_USDOT_Funds,Other_Federal_Funds
0,Alpine County Community Development,NaN,2022,Capital,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alpine County Community Development,NaN,2022,Operating,75944,75944,3448.0,24496.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amador Transit,NaN,2022,Capital,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [335]:
#### Combine this year and last year's data into 1 dataset.

# First verify that all the columns are the same - they ARE
r2023 = rr20_financial.columns
# r2023
r2022 = rr20_financial_2022.columns
difference = list(set(r2023) - set(r2022))
difference

allyears = pd.concat([rr20_financial, rr20_financial_2022], ignore_index = True)
print(allyears.shape)
allyears.head()

(172, 46)


,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Operating_Capital,Total_Annual_Revenues_Expended,Total_Annual_Expenses_by_Mode,Fare_Revenues,Other_Directly_Generated_Funds,Revenues_Accrued_Through_a_PT_Agreement,NonFederal_Funds,FTA_Metropolitan_Planning_5303,FTA_Urbanized_Area_Formula_Program_5307,FTA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,ARRA_Urbanized_Area_Program_Funds_5307,ARRA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,CARES_Act_Urbanized_Area_Program_Funds_5307,CRRSA_Act_Urbanized_Area_Program_Funds_5307,American_Rescue_Plan_Act_of_2021_Urbanized_Area_Program_Funds_5307,FTA_Clean_Fuels_Program_5308,FTA_Capital_Investment_Grants_5309,ARRA_Major_Capital_Investment_New_Starts_Funds_5309,American_Rescue_Plan_Act_of_2021_Fixed_Guideway_Capital_Investment_Grants_5303,FTA_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Formula_Program_5310,Capital_Assistance_Spent_on_Operations_5310,CRRSA_Act_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,American_Rescue_Plan_Act_of_2021_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,FTA_Formula_Grants_for_Rural_Areas_5311,Capital_Assistance_Spent_on_Operations_5311,FTA_ARRA_Other_than_Urbanized_Area_Program_Funds_5311,FTA_ARRA_Capital_Assistance_Spent_on_Operations_including_maintenance_expenses_5311,FTA_Tribal_Transit_Funds_5321,ARRA_Tribal_Transit_Funds_5311,CARES_Act_Rural_Area_Program_Funds_5311,CARES_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5311,CRRSA_Act_Rural_Area_Program_Funds_5311,CRRSA_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,American_Rescue_Plan_Act_of_2021_Rural_Area_Program_Funds_5311,American_Rescue_Plan_Act_of_2021_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,FTA_Job_Access_and_Reverse_Commute_Formula_Program_5316,State_of_Good_Repair_5308,FTA_Bus_and_Bus_Facilities,ARRA_TIGGER_Greenhouse_Gas_and_Energy_Reduction,Other_FTA_Funds,Other_USDOT_Funds,Other_Federal_Funds,date_uploaded
0,County of Sacramento Department of Transportation,None,2023,Capital,0,0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-10-11
1,County of Sacramento Department of Transportation,None,2023,Operating,2152007,2152007,108270.0,NaN,NaN,1302377.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,420860.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,320500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-10-11
2,Amador Transit,None,2023,Capital,0,0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2023-10-11
3,Amador Transit,None,2023,Operating,1484540,1484440,82216.0,5282.0,NaN,671952.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,56484.0,239425.0,NaN,NaN,NaN,NaN,NaN,226345.0,NaN,202836.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2023-10-11
4,San Benito County Local Transportation Authority,None,2023,Capital,180873,180873,0.0,0.0,NaN,180873.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-10-11


### VALIDATION CHECK DEVELOPMENT

In [336]:
numeric_columns = allyears.select_dtypes(include=['number']).columns
allyears[numeric_columns] = allyears[numeric_columns].fillna(0)

In [271]:
pd.set_option('display.max_columns', 1000); pd.set_option('display.width', 1000)

### for testing
df = allyears.copy()

agency = 'Tehama County Transit Agency'
agency_df = df[(df['Organization_Legal_Name'] == agency) & (df['Fiscal_Year'] == this_year)]
agency_df

,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Operating_Capital,Total_Annual_Revenues_Expended,Total_Annual_Expenses_by_Mode,Fare_Revenues,Other_Directly_Generated_Funds,Revenues_Accrued_Through_a_PT_Agreement,NonFederal_Funds,FTA_Metropolitan_Planning_5303,FTA_Urbanized_Area_Formula_Program_5307,FTA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,ARRA_Urbanized_Area_Program_Funds_5307,ARRA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,CARES_Act_Urbanized_Area_Program_Funds_5307,CRRSA_Act_Urbanized_Area_Program_Funds_5307,American_Rescue_Plan_Act_of_2021_Urbanized_Area_Program_Funds_5307,FTA_Clean_Fuels_Program_5308,FTA_Capital_Investment_Grants_5309,ARRA_Major_Capital_Investment_New_Starts_Funds_5309,American_Rescue_Plan_Act_of_2021_Fixed_Guideway_Capital_Investment_Grants_5303,FTA_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Formula_Program_5310,Capital_Assistance_Spent_on_Operations_5310,CRRSA_Act_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,American_Rescue_Plan_Act_of_2021_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,FTA_Formula_Grants_for_Rural_Areas_5311,Capital_Assistance_Spent_on_Operations_5311,FTA_ARRA_Other_than_Urbanized_Area_Program_Funds_5311,FTA_ARRA_Capital_Assistance_Spent_on_Operations_including_maintenance_expenses_5311,FTA_Tribal_Transit_Funds_5321,ARRA_Tribal_Transit_Funds_5311,CARES_Act_Rural_Area_Program_Funds_5311,CARES_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5311,CRRSA_Act_Rural_Area_Program_Funds_5311,CRRSA_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,American_Rescue_Plan_Act_of_2021_Rural_Area_Program_Funds_5311,American_Rescue_Plan_Act_of_2021_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,FTA_Job_Access_and_Reverse_Commute_Formula_Program_5316,State_of_Good_Repair_5308,FTA_Bus_and_Bus_Facilities,ARRA_TIGGER_Greenhouse_Gas_and_Energy_Reduction,Other_FTA_Funds,Other_USDOT_Funds,Other_Federal_Funds,date_uploaded
14,Tehama County Transit Agency,TRAX,2023,Capital,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2023-09-29
15,Tehama County Transit Agency,TRAX,2023,Operating,1981850,1981850,0.0,0.0,0.0,1275496.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,37674.0,NaN,398903.0,NaN,0.0,0.0,0.0,NaN,269777.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2023-09-29


In [291]:
variable = 'Total_Annual_Expenses_by_Mode'

# Total_Annual_Expenses_by_Mode for Capital expenses
(agency_df[agency_df['Operating_Capital']=='Capital'][variable]).values[0]

# Has to equal the sum of all values entered for Directly Generated, Non-Federal and Federal Government Funds for Uses of Capital.
# so the sum of all columns from 'Other_Directly_Generated_Funds' to the end of the row. Verified by looking at NTD error reports and the 2022 RR-20 data. 
start_index = df.columns.get_loc('Other_Directly_Generated_Funds') #7

df_capital_finances = agency_df[agency_df['Operating_Capital']=='Capital']

(df_capital_finances.iloc[:, start_index:len(agency_df.columns)]
.sum(numeric_only=True, axis=1).values[0] 
)


0.0

In [293]:
def rr20f_001c(df, this_year):
    agencies = df[df['Fiscal_Year']==this_year]['Organization_Legal_Name'].unique()
    output = []

    for agency in agencies:
        agency_df = df[(df['Organization_Legal_Name'] == agency) & (df['Fiscal_Year'] == this_year)]
        df_capital_finances = agency_df[agency_df['Operating_Capital']=='Capital']
        
        if (len(agency_df) == 0):
            print(f"There is no data for {agency}")
            continue
        
        sum_a = df_capital_finances['Total_Annual_Expenses_by_Mode'].values[0]
        start_index = df.columns.get_loc('Other_Directly_Generated_Funds')
        sum_b = (df_capital_finances.iloc[:, start_index:len(agency_df.columns)]
                 .sum(numeric_only=True, axis=1).values[0])
        
        if round(sum_a) == round(sum_b):
            result = "pass"
            check_name = "RR20F-001C: equal totals for capital expenses by mode and funding source expenditures"
            description = ""
        elif round(sum_a) != round(sum_b):
            result = "fail"
            check_name = f"RR20F-001C: equal totals for capital expenses by mode and funding source expenditures"
            description = f"The sum of Total Expenses for all modes for Uses of Capital {sum_a} does not equal the sum of all values entered for Directly Generated, Non-Federal and Federal Government Funds {sum_b} for Uses of Capital. Please revise or explain."
        
        output_line = {"Organization": agency,
               "name_of_check" : check_name,
                "value_checked": f"Total_Annual_Expenses_by_Mode = {sum_a},by funding source = {sum_b}",
                "check_status": result,
                "Description": description}

        output.append(output_line)
    checks = pd.DataFrame(output).sort_values(by="Organization")
    return checks


In [296]:
df = allyears.copy() #recopying the whole dataset here to eed into the function. In above cells we cut it down to a single agency.
v_cap_expenses = rr20f_001c(df, this_year)
v_cap_expenses

,Organization,name_of_check,value_checked,check_status,Description
4,Alpine County Community Development,RR20F-001C: equal totals for capital expenses ...,"Total_Annual_Expenses_by_Mode = 0,by funding s...",pass,
1,Amador Transit,RR20F-001C: equal totals for capital expenses ...,"Total_Annual_Expenses_by_Mode = 0,by funding s...",pass,
21,City of Arvin,RR20F-001C: equal totals for capital expenses ...,"Total_Annual_Expenses_by_Mode = 0,by funding s...",pass,
8,City of Auburn,RR20F-001C: equal totals for capital expenses ...,"Total_Annual_Expenses_by_Mode = 273426,by fund...",pass,
30,City of California City,RR20F-001C: equal totals for capital expenses ...,"Total_Annual_Expenses_by_Mode = 0,by funding s...",pass,
16,City of Corcoran - Corcoran Area Transit,RR20F-001C: equal totals for capital expenses ...,"Total_Annual_Expenses_by_Mode = 0,by funding s...",pass,
31,City of Dixon,RR20F-001C: equal totals for capital expenses ...,"Total_Annual_Expenses_by_Mode = 0,by funding s...",pass,
9,City of Guadalupe,RR20F-001C: equal totals for capital expenses ...,"Total_Annual_Expenses_by_Mode = 0,by funding s...",pass,
13,City of McFarland,RR20F-001C: equal totals for capital expenses ...,"Total_Annual_Expenses_by_Mode = 0,by funding s...",pass,
3,City of Needles,RR20F-001C: equal totals for capital expenses ...,"Total_Annual_Expenses_by_Mode = 0,by funding s...",pass,


In [242]:
df = allyears.copy() #recopying the whole dataset here to read into the function. In above cells we cut it down to a single agency.

def financial_checks(df, variable, this_year, last_year):
    agencies = df[df['Fiscal_Year']==this_year]['Organization_Legal_Name'].unique()
    output = []

    for agency in agencies:
        if (len(df[(df['Organization_Legal_Name']==agency) & (df['Fiscal_Year']==this_year)]) == 0):
            print(f"There is no data for {agency}")
            continue
        
        ### combine operating/capital rows into sums
        value_thisyr = (round(df[(df['Organization_Legal_Name'] == agency) 
                              & (df['Fiscal_Year'] == this_year)]
                      [variable].unique().sum()))

        if (value_thisyr == 0) and (variable == 'FTA_Formula_Grants_for_Rural_Areas_5311'):
            result = "fail"
            check_name = "RR20F-070: no funds"
            description = (f"The §5311 program is not listed as a revenue source in your report in {this_year}, please provide a narrative justification.")
        elif (round(value_thisyr) % 1000 == 0) and (value_thisyr !=0):
            result = "fail"
            check_name = f"Rounded to thousand: {variable}"
            description = (f"{variable} is rounded to the nearest thousand, but should be reported as exact values. Please provide a narrative justification.")

            
        # Check whether data for last year is also present, if so perform prior yr comparisons.
        if len(df[(df['Organization_Legal_Name']==agency) & (df['Fiscal_Year']==last_year)]) == 0:
            value_lastyr = None
            continue

        value_lastyr = (round(df[(df['Organization_Legal_Name'] == agency) 
                              & (df['Fiscal_Year'] == last_year)]
                      [variable].unique().sum()))

        if ((round(value_thisyr)==0 and round(value_lastyr) != 0) | (round(value_thisyr)!=0 and round(value_lastyr) == 0)) and (variable != 'Other_Directly_Generated_Funds'):
            result = "fail"
            check_name = f"Change from 0: {variable}"
            description = f"{variable} funding changed either from or to zero compared to last year. Please provide a narrative justification."
        elif (abs(round(value_lastyr)) == abs(round(value_thisyr))) and (value_thisyr !=0) and (value_lastyr !=0):
            result = "fail"
            check_name = f"Same value: {variable}"
            description = (f"You have identical values for {variable} reported in {this_year} and {last_year}, which is unusual. Please provide a narrative justification.")
        else:
            result = "pass"
            check_name = f"{variable}"
            description = ""
            
        output_line = {"Organization": agency,
               "name_of_check" : check_name,
                "value_checked": f"{this_year} = {value_thisyr}, {last_year} = {value_lastyr}",
                "check_status": result,
                "Description": description}

        output.append(output_line)
    checks = pd.DataFrame(output).sort_values(by="Organization")
    return checks

In [243]:
# financial_checks(allyears, 'FTA_Formula_Grants_for_Rural_Areas_5311', this_year, last_year)

v_5311 = financial_checks(allyears, 'FTA_Formula_Grants_for_Rural_Areas_5311', this_year, last_year)
v_odg = financial_checks(allyears, 'Other_Directly_Generated_Funds', this_year, last_year)
v_farerev = financial_checks(allyears, 'Fare_Revenues', this_year, last_year)
# Need to run checks on "Local_Funds" once we figure out where that data is.

In [244]:
f_checks = pd.concat([v_5311, v_odg, v_farerev], ignore_index=True).sort_values(by="Organization")
f_checks

,Organization,name_of_check,value_checked,check_status,Description
0,City of Arvin,FTA_Formula_Grants_for_Rural_Areas_5311,"2023 = 76034, 2022 = 73230",pass,
5,City of Arvin,Other_Directly_Generated_Funds,"2023 = 18177, 2022 = 0",pass,
10,City of Arvin,Fare_Revenues,"2023 = 42028, 2022 = 36386",pass,
1,City of Corcoran - Corcoran Area Transit,FTA_Formula_Grants_for_Rural_Areas_5311,"2023 = 377292, 2022 = 165499",pass,
6,City of Corcoran - Corcoran Area Transit,Other_Directly_Generated_Funds,"2023 = 0, 2022 = 0",pass,
11,City of Corcoran - Corcoran Area Transit,Fare_Revenues,"2023 = 45715, 2022 = 39478",pass,
2,County of Shasta Department of Public Works,FTA_Formula_Grants_for_Rural_Areas_5311,"2023 = 728820, 2022 = 373943",pass,
7,County of Shasta Department of Public Works,Other_Directly_Generated_Funds,"2023 = 0, 2022 = 0",pass,
12,County of Shasta Department of Public Works,Fare_Revenues,"2023 = 0, 2022 = 0",pass,
3,Plumas County Transportation Commission,FTA_Formula_Grants_for_Rural_Areas_5311,"2023 = 369904, 2022 = 550642",pass,


In [245]:
def equal_totals(this_year, df):
    agencies = df['Organization_Legal_Name'].unique()
    output = []

    for agency in agencies:
        agency_df = df[(df['Organization_Legal_Name']==agency) & (df['Fiscal_Year']==this_year)].drop_duplicates()
        
        if len(agency_df) > 0:
            Total_Annual_Revenues_Expended = agency_df[agency_df['Operating_Capital']=='Operating']['Total_Annual_Revenues_Expended'].values[0]
            Total_Annual_Expenses_by_Mode = agency_df[agency_df['Operating_Capital']=='Operating']['Total_Annual_Expenses_by_Mode'].values[0]

            if round(Total_Annual_Revenues_Expended) != round(Total_Annual_Expenses_by_Mode):
                result = "fail"
                check_name = "RR20F-001OA: equal totals"
                description = f"Total_Annual_Revenues_Expended (${Total_Annual_Revenues_Expended}) should, but does not, equal Total_Annual_Expenses_by_Mode (${Total_Annual_Expenses_by_Mode}). Please provide a narrative justification."
            else:
                result = "pass"
                check_name = "RR20F-001OA: equal totals"
                description = ""
            
            output_line = {"Organization": agency,
                       "name_of_check" : check_name,
                        "value_checked": f"Total_Annual_Revenues_Expended = ${Total_Annual_Revenues_Expended},Total_Annual_Expenses_by_Mode = ${Total_Annual_Expenses_by_Mode}",
                        "check_status": result,
                        "Description": description}
            output.append(output_line)
            
        else:
            pass

    checks = pd.DataFrame(output).sort_values(by="Organization")
    return checks
    
                                                                                                

In [248]:
equ = equal_totals(this_year, allyears)
f_checks  = pd.concat([f_checks, equ], ignore_index=True).sort_values(by="Organization")
f_checks

,Organization,name_of_check,value_checked,check_status,Description
0,City of Arvin,FTA_Formula_Grants_for_Rural_Areas_5311,"2023 = 76034, 2022 = 73230",pass,
1,City of Arvin,Other_Directly_Generated_Funds,"2023 = 18177, 2022 = 0",pass,
2,City of Arvin,Fare_Revenues,"2023 = 42028, 2022 = 36386",pass,
15,City of Arvin,RR20F-001OA: equal totals,"Total_Annual_Revenues_Expended = $982115,Total...",fail,Total_Annual_Revenues_Expended ($982115) shoul...
3,City of Corcoran - Corcoran Area Transit,FTA_Formula_Grants_for_Rural_Areas_5311,"2023 = 377292, 2022 = 165499",pass,
4,City of Corcoran - Corcoran Area Transit,Other_Directly_Generated_Funds,"2023 = 0, 2022 = 0",pass,
5,City of Corcoran - Corcoran Area Transit,Fare_Revenues,"2023 = 45715, 2022 = 39478",pass,
16,City of Corcoran - Corcoran Area Transit,RR20F-001OA: equal totals,"Total_Annual_Revenues_Expended = $884978,Total...",pass,
6,County of Shasta Department of Public Works,FTA_Formula_Grants_for_Rural_Areas_5311,"2023 = 728820, 2022 = 373943",pass,
7,County of Shasta Department of Public Works,Other_Directly_Generated_Funds,"2023 = 0, 2022 = 0",pass,


#### This section for testing the above function
Un-uncomment the code blocks with `Cmd` + `/` to run them.

In [125]:
# allyears[allyears['Fiscal_Year']==2023]['Organization_Legal_Name'].unique()

array(['City of Arvin', 'City of Corcoran - Corcoran Area Transit',
       'County of Shasta Department of Public Works',
       'Plumas County Transportation Commission',
       'Tehama County Transit Agency'], dtype=object)

In [126]:
# agency = 'Tehama County Transit Agency'

# agency_df = allyears[(allyears['Organization_Legal_Name'] == agency) & (agency_df['Fiscal_Year']==2023)]
# print(len(agency_df))
# agency_df


0


,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Operating_Capital,Total_Annual_Revenues_Expended,Total_Annual_Expenses_by_Mode,Fare_Revenues,Other_Directly_Generated_Funds,Revenues_Accrued_Through_a_PT_Agreement,NonFederal_Funds,FTA_Metropolitan_Planning_5303,FTA_Urbanized_Area_Formula_Program_5307,FTA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,ARRA_Urbanized_Area_Program_Funds_5307,ARRA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,CARES_Act_Urbanized_Area_Program_Funds_5307,CRRSA_Act_Urbanized_Area_Program_Funds_5307,American_Rescue_Plan_Act_of_2021_Urbanized_Area_Program_Funds_5307,FTA_Clean_Fuels_Program_5308,FTA_Capital_Investment_Grants_5309,ARRA_Major_Capital_Investment_New_Starts_Funds_5309,American_Rescue_Plan_Act_of_2021_Fixed_Guideway_Capital_Investment_Grants_5303,FTA_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Formula_Program_5310,Capital_Assistance_Spent_on_Operations_5310,CRRSA_Act_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,American_Rescue_Plan_Act_of_2021_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,FTA_Formula_Grants_for_Rural_Areas_5311,Capital_Assistance_Spent_on_Operations_5311,FTA_ARRA_Other_than_Urbanized_Area_Program_Funds_5311,FTA_ARRA_Capital_Assistance_Spent_on_Operations_including_maintenance_expenses_5311,FTA_Tribal_Transit_Funds_5321,ARRA_Tribal_Transit_Funds_5311,CARES_Act_Rural_Area_Program_Funds_5311,CARES_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5311,CRRSA_Act_Rural_Area_Program_Funds_5311,CRRSA_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,American_Rescue_Plan_Act_of_2021_Rural_Area_Program_Funds_5311,American_Rescue_Plan_Act_of_2021_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,FTA_Job_Access_and_Reverse_Commute_Formula_Program_5316,State_of_Good_Repair_5308,FTA_Bus_and_Bus_Facilities,ARRA_TIGGER_Greenhouse_Gas_and_Energy_Reduction,Other_FTA_Funds,Other_USDOT_Funds,Other_Federal_Funds


In [100]:
# agency_df[(agency_df['Operating_Capital']=='Operating') & (agency_df['Fiscal_Year']==2023)]['Total_Annual_Revenues_Expended'].values[0]


982115

In [101]:
# agency_df[(agency_df['Operating_Capital']=='Operating') & (agency_df['Fiscal_Year']==2023)]['Total_Annual_Expenses_by_Mode'].values[0]

963938

### RR20F-001C

In [ ]:
### ------- RR20F-001C
# Has to equal the sum of all values entered for Directly Generated, Non-Federal and Federal Government Funds for Uses of Capital.
# so the sum of all columns from 'Other_Directly_Generated_Funds' to the end of the row. Verified by looking at NTD error reports and the 2022 RR-20 data. 
start_index = df.columns.get_loc('Other_Directly_Generated_Funds') #7

df_capital_finances = agency_df[agency_df['Operating_Capital']=='Capital']

(df_capital_finances.iloc[:, start_index:len(agency_df.columns)]
.sum(numeric_only=True, axis=1).values[0] 
)

### RR20F-182. Validation check for whether any new fleet's in-service date equates with no capital expenses
* if there is any new fleet vehicle, there must be capital expenses reported.

In [321]:
# Run validation checks against vehicle inventory
veh_inv_query = f"""SELECT * FROM `cal-itp-data-infra.blackcat_raw.{this_year}_revenue_vehicle_inventory`"""
veh_inv = client.query(veh_inv_query).to_dataframe()
orgs_query = f"""SELECT * FROM `cal-itp-data-infra.blackcat_raw.{this_year}_organizations`"""
orgs = client.query(orgs_query).to_dataframe()

In [322]:
veh_inv.head()

,Organization,VIN,LicensePlate,AgencyId,DOTId,Funded_By_DOT,Funding_Program,Grant_Primary,Grant_Secondary,DOT_Contract,Purchase_Order_Number,Purchase_Order_Date,Invoice_Number,Payment_Voucher_Number,Payment_Voucher_Date,Award_Year,UPIN,Total_Cost,Federal_Share,Federal_Share_Percent,State_Share,State_Share_Percent,Local_Share,Local_Share_Percent,DOT_Capital_Responsibility,Organization_Capital_Responsibility,Other_Capital_Responsibility,Other_Capital_Responsibility_Org,Category,Vehicle_Year,Manufacturer,Model,Chassis,Fuel_Type,Vehicle_Height_Feet,Vehicle_Height_Inches,Vehicle_Length_Feet,GVWR,Seating_Capacity,Standing_Capacity,Wheelchair_Positions,Wheelchair_Ramp_Or_Lift,Wheelchair_Ramp_Or_Lift_Manufacturer,Emergency_Contingency_Vehicle,Status,Improvement_Purpose,Current_Condition,Current_Condition_Date,Condition_Rating,Current_Mileage,Current_Mileage_Date,Year_Of_Renewal,Renewal_Type,Price_Agreement_Number,_Price_Agreement_Date,Purchase_Date,Vendor,Received_Date,Last_Inspection_Date,Vehicle_Location_Street,Vehicle_Location_City,Vehicle_Location_State,In_Service_Date,Useful_Life_Years_Remaining,Useful_Life_Miles_Remaining,Out_Of_Service_Date,Is_DOT_LienHolder,Is_Title_On_File_With_DOT,Title_Number,Registration_Number,NTD_ID,Vehicle_Type,Average_Estimated_Service_Years_When_New,Dedicated_Fleet,Ownership_Type,Funding_Source,NTD_Other_Mode,NTD_Other_Type_Of_Service,Vehicle_Service_Modes_Comma_Delimited,Comments,date_uploaded
0,3.00,1FD3E35LX8DB57263,None,None,NaN,Yes,Section 5316,CA-37-X069,None,646611,None,NaT,NaN,NaN,NaN,NaN,None,53000.0,42400.0,NaN,NaN,NaN,10600.0,NaN,NaN,NaN,NaN,None,Light-Duty Mid-Sized Bus,2009,Gillig Corporation,None,Ford E-350,CN - Compressed Natural Gas,NaN,NaN,30.0,NaN,NaN,NaN,NaN,None,None,None,Active,Expansion (New),NaN,NaT,None,NaN,NaT,NaN,None,None,NaN,2009-02-18,None,NaT,NaT,None,None,None,NaT,NaN,NaN,NaT,No,None,None,None,None,CU - Cutaway Bus,NaN,None,None,None,None,None,None,None,2023-10-13
1,3.00,1FD3E35LX8DB57264,None,None,NaN,Yes,Section 5316,CA-37-X069,None,646611,None,NaT,NaN,NaN,NaN,NaN,None,53000.0,42400.0,NaN,NaN,NaN,10600.0,NaN,NaN,NaN,NaN,None,Light-Duty Mid-Sized Bus,2009,Gillig Corporation,None,Ford E-350,CN - Compressed Natural Gas,NaN,NaN,30.0,NaN,NaN,NaN,NaN,None,None,None,Active,Expansion (New),NaN,NaT,None,NaN,NaT,NaN,None,None,NaN,2009-02-18,None,NaT,NaT,None,None,None,NaT,NaN,NaN,NaT,No,None,None,None,None,CU - Cutaway Bus,NaN,None,None,None,None,None,None,None,2023-10-13
2,Able Industries,1FDEE3FL7FDA08586,43980V1,None,NaN,Yes,5310 Trad,None,None,None,None,NaT,NaN,NaN,NaN,NaN,None,60000.0,53118.0,0.8853,6882.0,0.1147,NaN,NaN,NaN,NaN,NaN,None,Heavy-Duty Small Bus,2015,Starcraft,Starlite,Ford E-350,GA - Gasoline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lift,Braun,Yes,Active,Replacement,NaN,NaT,None,NaN,NaT,NaN,None,None,NaN,NaT,None,NaT,2016-09-14,8929 W. Goshen Ave.,Visalia,CA,2015-08-03,8.0,350000.0,NaT,Yes,None,None,None,None,CU - Cutaway Bus,10.0,None,None,None,None,None,"Demand Response, Bus",None,2023-10-13
3,Alchemia,523MF1A60CM101085,None,None,NaN,Yes,Section 5317,CA-57-X053,None,640721,None,NaT,NaN,NaN,NaN,NaN,None,47746.0,38197.0,NaN,NaN,NaN,9549.0,NaN,NaN,NaN,NaN,None,"Light-Duty Vans, Sedans or Buses",2012,Vehicle Production Group,MV-1,None,GA - Gasoline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,Active,Expansion (New),NaN,NaT,None,NaN,NaT,NaN,None,None,NaN,NaT,None,NaT,NaT,None,None,None,NaT,NaN,NaN,NaT,No,None,None,None,None,BU - Bus,NaN,None,None,None,None,None,None,None,2023-10-13
4,Amador Transit,1GB6G5BL2D1114841,1391917,404,NaN,Yes,5310 Trad,None,None,None,641503,NaT,NaN,NaN,NaN,NaN,None,99786.0,88341.0,0.8853,NaN,NaN,11445.0,NaN,NaN,NaN,NaN,None,Medium-Duty Bus,2013,Chevrolet Motor Division - GMC,Glaval,GM 4500,DL - Diesel Low-sulfur,10.0,5.0,24.0,10550.0,17.0,NaN,2.0,Lift,Braun,Yes,Disposed,Replacement,4.0,2019-08-15 18:40:55.647,Marginal,141247.0,2019-06-28,2019.0,None,None,NaN,NaT,A to Z Bus Sales,2013-08-23,2017-11-22,11400 American Legion Dr.,JACKSON,CA,2013-09-01,3.0

In [384]:
# There are more organizations in the inventory than needed subrecipients. Cut down to only what we want. 
veh_inv_orgs = (veh_inv.merge(orgs, on = 'Organization', 
                            indicator=True)
                .query('_merge == "both"').drop(columns=['_merge', 'date_uploaded_x', 'date_uploaded_y']))
print(veh_inv_orgs.shape)
veh_inv_orgs.head()

(2599, 80)


,Organization,VIN,LicensePlate,AgencyId,DOTId,Funded_By_DOT,Funding_Program,Grant_Primary,Grant_Secondary,DOT_Contract,Purchase_Order_Number,Purchase_Order_Date,Invoice_Number,Payment_Voucher_Number,Payment_Voucher_Date,Award_Year,UPIN,Total_Cost,Federal_Share,Federal_Share_Percent,State_Share,State_Share_Percent,Local_Share,Local_Share_Percent,DOT_Capital_Responsibility,Organization_Capital_Responsibility,Other_Capital_Responsibility,Other_Capital_Responsibility_Org,Category,Vehicle_Year,Manufacturer,Model,Chassis,Fuel_Type,Vehicle_Height_Feet,Vehicle_Height_Inches,Vehicle_Length_Feet,GVWR,Seating_Capacity,Standing_Capacity,Wheelchair_Positions,Wheelchair_Ramp_Or_Lift,Wheelchair_Ramp_Or_Lift_Manufacturer,Emergency_Contingency_Vehicle,Status,Improvement_Purpose,Current_Condition,Current_Condition_Date,Condition_Rating,Current_Mileage,Current_Mileage_Date,Year_Of_Renewal,Renewal_Type,Price_Agreement_Number,_Price_Agreement_Date,Purchase_Date,Vendor,Received_Date,Last_Inspection_Date,Vehicle_Location_Street,Vehicle_Location_City,Vehicle_Location_State,In_Service_Date,Useful_Life_Years_Remaining,Useful_Life_Miles_Remaining,Out_Of_Service_Date,Is_DOT_LienHolder,Is_Title_On_File_With_DOT,Title_Number,Registration_Number,NTD_ID,Vehicle_Type,Average_Estimated_Service_Years_When_New,Dedicated_Fleet,Ownership_Type,Funding_Source,NTD_Other_Mode,NTD_Other_Type_Of_Service,Vehicle_Service_Modes_Comma_Delimited,Comments
0,Amador Transit,1GB6G5BL2D1114841,1391917,404,NaN,Yes,5310 Trad,None,None,None,641503,NaT,NaN,NaN,NaN,NaN,None,99786.0,88341.0,0.8853,NaN,NaN,11445.0,NaN,NaN,NaN,NaN,None,Medium-Duty Bus,2013,Chevrolet Motor Division - GMC,Glaval,GM 4500,DL - Diesel Low-sulfur,10.0,5.0,24.0,10550.0,17.0,NaN,2.0,Lift,Braun,Yes,Disposed,Replacement,4.0,2019-08-15 18:40:55.647,Marginal,141247.0,2019-06-28,2019.0,None,None,NaN,NaT,A to Z Bus Sales,2013-08-23,2017-11-22,11400 American Legion Dr.,JACKSON,CA,2013-09-01,3.0,87903.0,NaT,Yes,Yes,None,None,None,CU - Cutaway Bus,10.0,Yes,Owned outright by public agency (OOPA),EMSID - Enhanced Mobility of Seniors & Individ...,None,None,"Deviated Fixed Route, Demand Response",This vehicle is permanently out of service as ...
1,Amador Transit,1FDWE45F02HB76062,1130030,27,NaN,Yes,Section 5316,None,None,None,None,NaT,NaN,NaN,NaN,NaN,None,75000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Medium-Duty Bus,2002,Ford Motor Corporation,Goshen,Ford E-450,DL - Diesel Low-sulfur,NaN,NaN,25.0,10550.0,20.0,NaN,4.0,Lift,Braun,Yes,Disposed,Replacement,2.0,2018-03-01 00:00:00.000,Poor,307852.0,2017-11-06,2010.0,None,None,NaN,NaT,None,NaT,2017-11-06,11400 American Legion Dr.,Jackson,CA,2003-03-31,-10.0,-107852.0,NaT,No,No,None,None,None,CU - Cutaway Bus,10.0,Yes,Owned outright by public agency (OOPA),RAFP - Rural Area Formula Program,None,None,"Deviated Fixed Route, Demand Response",This vehicle has sold through Auction and is n...
2,Amador Transit,1GBGSV1G26F413492,1241544,502,NaN,Yes,Section 5311,None,None,None,None,NaT,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Medium-Duty Bus,2006,Chevrolet Motor Division - GMC,El Dorado,GM 5500,GA - Gasoline,10.0,NaN,32.0,22000.0,28.0,NaN,2.0,Lift,Braun,Yes,Disposed,Replacement,0.1,2017-12-08 00:00:00.000,Poor,169913.0,NaT,2018.0,None,None,NaN,NaT,None,NaT,NaT,None,None,None,NaT,-4.0,30087.0,2016-03-21,No,None,None,None,None,CU - Cutaway Bus,10.0,No,Owned outright by public agency (OOPA),None,None,None,Deviated Fixed Route,Vehicle to be Auctioned this month and removed...
3,Amador Transit,1GBGSV1G46F412960,1241551,501,NaN,Yes,Section 5316,None,None,None,None,NaT,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Medium-Duty Bus,2006,Chevrolet Motor Division - GMC,El Dorado,GM 5500,GA - Gasoline,10.0,NaN,32.0,22000.0,28.0,NaN,2.0,Lift,Braun,Yes,Disposed,Replacement,0.1,2017-12-08 00:00:00.000,Poor,214913.0,2017-11-30,2018.0,None,None,NaN,NaT,None,NaT,2017-11-09,11400 American Legion Dr.,JACKSON,CA,NaT,-6.0,-14913.0,NaT,No,No,None,None,None,CU - Cut

In [325]:
df = veh_inv_orgs.copy()

agency = 'Tehama County Transit Agency'
agency_df = df[(df['Organization'] == agency)]
agency_df

,Organization,VIN,LicensePlate,AgencyId,DOTId,Funded_By_DOT,Funding_Program,Grant_Primary,Grant_Secondary,DOT_Contract,Purchase_Order_Number,Purchase_Order_Date,Invoice_Number,Payment_Voucher_Number,Payment_Voucher_Date,Award_Year,UPIN,Total_Cost,Federal_Share,Federal_Share_Percent,State_Share,State_Share_Percent,Local_Share,Local_Share_Percent,DOT_Capital_Responsibility,Organization_Capital_Responsibility,Other_Capital_Responsibility,Other_Capital_Responsibility_Org,Category,Vehicle_Year,Manufacturer,Model,Chassis,Fuel_Type,Vehicle_Height_Feet,Vehicle_Height_Inches,Vehicle_Length_Feet,GVWR,Seating_Capacity,Standing_Capacity,Wheelchair_Positions,Wheelchair_Ramp_Or_Lift,Wheelchair_Ramp_Or_Lift_Manufacturer,Emergency_Contingency_Vehicle,Status,Improvement_Purpose,Current_Condition,Current_Condition_Date,Condition_Rating,Current_Mileage,Current_Mileage_Date,Year_Of_Renewal,Renewal_Type,Price_Agreement_Number,_Price_Agreement_Date,Purchase_Date,Vendor,Received_Date,Last_Inspection_Date,Vehicle_Location_Street,Vehicle_Location_City,Vehicle_Location_State,In_Service_Date,Useful_Life_Years_Remaining,Useful_Life_Miles_Remaining,Out_Of_Service_Date,Is_DOT_LienHolder,Is_Title_On_File_With_DOT,Title_Number,Registration_Number,NTD_ID,Vehicle_Type,Average_Estimated_Service_Years_When_New,Dedicated_Fleet,Ownership_Type,Funding_Source,NTD_Other_Mode,NTD_Other_Type_Of_Service,Vehicle_Service_Modes_Comma_Delimited,Comments,date_uploaded_x,date_uploaded_y
1136,Tehama County Transit Agency,1F66F5DN3L0A04899,1612911,T-2,NaN,Yes,Section 5311,CA-18-X069 | 0015000075,None,None,None,NaT,NaN,NaN,NaN,NaN,None,238579.00,238579.00,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Fixed Guideway Trolley (steel-wheeled),2020,Hometown,Villager,None,GA - Gasoline,NaN,NaN,29.0,NaN,24.0,NaN,NaN,Lift,Braun,None,Active,Replacement,5.0,2021-04-16,Excellent,111340.0,2023-08-31,2027.0,None,None,NaN,NaT,None,NaT,NaT,None,None,None,NaT,22.0,-111340.0,NaT,Yes,None,None,None,None,RT - Rubber Tired Vintage Trolley,14.0,Yes,Owned outright by public agency (OOPA),None,None,None,None,None,2023-10-13,2023-10-13
1137,Tehama County Transit Agency,1F66F5DN8L0A04896,1612910,T-1,NaN,Yes,Section 5311,CA-18-X069 | 0015000075,None,None,None,NaT,NaN,NaN,NaN,NaN,None,238579.00,211310.00,0.8857,27269.00,0.1142,NaN,NaN,NaN,NaN,NaN,None,Fixed Guideway Trolley (steel-wheeled),2020,Hometown,Villager,None,GA - Gasoline,NaN,NaN,29.0,NaN,24.0,NaN,NaN,Lift,Braun,Yes,Active,Replacement,5.0,2021-03-25,Excellent,87111.0,2023-08-31,2027.0,None,None,NaN,NaT,None,NaT,NaT,None,None,None,2021-03-01,22.0,-87111.0,NaT,No,None,None,None,None,RT - Rubber Tired Vintage Trolley,14.0,Yes,Owned outright by public agency (OOPA),None,TB - Trolleybus,None,Trolleybus,None,2023-10-13,2023-10-13
1138,Tehama County Transit Agency,1FDES6PG1LKB31822,1613144,978,NaN,Yes,Section 5311,PTMISEA,None,None,None,NaT,NaN,NaN,NaN,NaN,None,89655.00,74014.00,0.8255,15641.00,0.1744,NaN,NaN,NaN,NaN,NaN,None,Medium-Duty Bus,2020,Ford Motor Corporation,None,350 HD,GA - Gasoline,NaN,NaN,21.0,NaN,8.0,NaN,2.0,Lift,Braun,Yes,Active,None,5.0,2020-12-04,Excellent,111642.0,2023-08-31,2027.0,None,None,NaN,NaT,None,NaT,NaT,None,None,None,2020-12-03,4.0,88358.0,NaT,No,None,None,None,None,CU - Cutaway Bus,10.0,None,Leased or borrowed from related parties by a p...,None,CB - Commuter Bus,None,Demand Response,None,2023-10-13,2023-10-13
1139,Tehama County Transit Agency,1FDES6PG6LKB13816,163153,979,NaN,Yes,Section 5311,CA-18-X069 | 0015000075,None,None,None,NaT,NaN,NaN,NaN,NaN,None,92015.00,81498.00,0.8857,10517.00,0.1142,92015.0,1.0,NaN,NaN,NaN,None,Medium-Duty Bus,2020,Ford Motor Corporation,350HD,None,GA - Gasoline,NaN,NaN,21.0,NaN,10.0,NaN,NaN,Lift,Braun,None,Active,Replacement,5.0,2021-03-25,Excellent,39012.0,2023-08-31,NaN,None,None,NaN,NaT,None,NaT,NaT,None,None,None,2020-12-07,4.0,160988.0,NaT,No,None,None,None,None,CU - Cutaway Bus,10.0,None,None,None,None,None,None,None,2023-10-13,2023-10-13
1140,Tehama County Transit Agency,1FDES6PG8LKB31820,161

In [341]:
df = allyears.copy()
variable = 'Total_Annual_Expenses_by_Mode'

agency_df = df[df['Organization_Legal_Name'] == agency]

# Total_Annual_Expenses_by_Mode for Capital expenses
(agency_df[agency_df['Operating_Capital']=='Capital'][variable]).values[0]


0

In [339]:
df.head(3)

,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Operating_Capital,Total_Annual_Revenues_Expended,Total_Annual_Expenses_by_Mode,Fare_Revenues,Other_Directly_Generated_Funds,Revenues_Accrued_Through_a_PT_Agreement,NonFederal_Funds,FTA_Metropolitan_Planning_5303,FTA_Urbanized_Area_Formula_Program_5307,FTA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,ARRA_Urbanized_Area_Program_Funds_5307,ARRA_Urbanized_Area_Program_Funds_Capital_Assistance_Spent_on_Operations_5307,CARES_Act_Urbanized_Area_Program_Funds_5307,CRRSA_Act_Urbanized_Area_Program_Funds_5307,American_Rescue_Plan_Act_of_2021_Urbanized_Area_Program_Funds_5307,FTA_Clean_Fuels_Program_5308,FTA_Capital_Investment_Grants_5309,ARRA_Major_Capital_Investment_New_Starts_Funds_5309,American_Rescue_Plan_Act_of_2021_Fixed_Guideway_Capital_Investment_Grants_5303,FTA_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Formula_Program_5310,Capital_Assistance_Spent_on_Operations_5310,CRRSA_Act_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,American_Rescue_Plan_Act_of_2021_Enhanced_Mobility_of_Seniors_and_Individuals_with_Disabilities_Program_Funds_5310,FTA_Formula_Grants_for_Rural_Areas_5311,Capital_Assistance_Spent_on_Operations_5311,FTA_ARRA_Other_than_Urbanized_Area_Program_Funds_5311,FTA_ARRA_Capital_Assistance_Spent_on_Operations_including_maintenance_expenses_5311,FTA_Tribal_Transit_Funds_5321,ARRA_Tribal_Transit_Funds_5311,CARES_Act_Rural_Area_Program_Funds_5311,CARES_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5311,CRRSA_Act_Rural_Area_Program_Funds_5311,CRRSA_Act_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,American_Rescue_Plan_Act_of_2021_Rural_Area_Program_Funds_5311,American_Rescue_Plan_Act_of_2021_Public_Transportation_on_Indian_Reservations_Program_Funds_5321,FTA_Job_Access_and_Reverse_Commute_Formula_Program_5316,State_of_Good_Repair_5308,FTA_Bus_and_Bus_Facilities,ARRA_TIGGER_Greenhouse_Gas_and_Energy_Reduction,Other_FTA_Funds,Other_USDOT_Funds,Other_Federal_Funds,date_uploaded
0,County of Sacramento Department of Transportation,None,2023,Capital,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-10-11
1,County of Sacramento Department of Transportation,None,2023,Operating,2152007,2152007,108270.0,0.0,0.0,1302377.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,420860.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,320500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-10-11
2,Amador Transit,None,2023,Capital,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-10-11


In [333]:
agency = 'Tehama County Transit Agency'
veh_df = veh_inv_orgs[(veh_inv_orgs['Organization'] == agency)]
veh_df

,Organization,VIN,LicensePlate,AgencyId,DOTId,Funded_By_DOT,Funding_Program,Grant_Primary,Grant_Secondary,DOT_Contract,Purchase_Order_Number,Purchase_Order_Date,Invoice_Number,Payment_Voucher_Number,Payment_Voucher_Date,Award_Year,UPIN,Total_Cost,Federal_Share,Federal_Share_Percent,State_Share,State_Share_Percent,Local_Share,Local_Share_Percent,DOT_Capital_Responsibility,Organization_Capital_Responsibility,Other_Capital_Responsibility,Other_Capital_Responsibility_Org,Category,Vehicle_Year,Manufacturer,Model,Chassis,Fuel_Type,Vehicle_Height_Feet,Vehicle_Height_Inches,Vehicle_Length_Feet,GVWR,Seating_Capacity,Standing_Capacity,Wheelchair_Positions,Wheelchair_Ramp_Or_Lift,Wheelchair_Ramp_Or_Lift_Manufacturer,Emergency_Contingency_Vehicle,Status,Improvement_Purpose,Current_Condition,Current_Condition_Date,Condition_Rating,Current_Mileage,Current_Mileage_Date,Year_Of_Renewal,Renewal_Type,Price_Agreement_Number,_Price_Agreement_Date,Purchase_Date,Vendor,Received_Date,Last_Inspection_Date,Vehicle_Location_Street,Vehicle_Location_City,Vehicle_Location_State,In_Service_Date,Useful_Life_Years_Remaining,Useful_Life_Miles_Remaining,Out_Of_Service_Date,Is_DOT_LienHolder,Is_Title_On_File_With_DOT,Title_Number,Registration_Number,NTD_ID,Vehicle_Type,Average_Estimated_Service_Years_When_New,Dedicated_Fleet,Ownership_Type,Funding_Source,NTD_Other_Mode,NTD_Other_Type_Of_Service,Vehicle_Service_Modes_Comma_Delimited,Comments,date_uploaded_x,date_uploaded_y
1136,Tehama County Transit Agency,1F66F5DN3L0A04899,1612911,T-2,NaN,Yes,Section 5311,CA-18-X069 | 0015000075,None,None,None,NaT,NaN,NaN,NaN,NaN,None,238579.00,238579.00,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Fixed Guideway Trolley (steel-wheeled),2020,Hometown,Villager,None,GA - Gasoline,NaN,NaN,29.0,NaN,24.0,NaN,NaN,Lift,Braun,None,Active,Replacement,5.0,2021-04-16,Excellent,111340.0,2023-08-31,2027.0,None,None,NaN,NaT,None,NaT,NaT,None,None,None,NaT,22.0,-111340.0,NaT,Yes,None,None,None,None,RT - Rubber Tired Vintage Trolley,14.0,Yes,Owned outright by public agency (OOPA),None,None,None,None,None,2023-10-13,2023-10-13
1137,Tehama County Transit Agency,1F66F5DN8L0A04896,1612910,T-1,NaN,Yes,Section 5311,CA-18-X069 | 0015000075,None,None,None,NaT,NaN,NaN,NaN,NaN,None,238579.00,211310.00,0.8857,27269.00,0.1142,NaN,NaN,NaN,NaN,NaN,None,Fixed Guideway Trolley (steel-wheeled),2020,Hometown,Villager,None,GA - Gasoline,NaN,NaN,29.0,NaN,24.0,NaN,NaN,Lift,Braun,Yes,Active,Replacement,5.0,2021-03-25,Excellent,87111.0,2023-08-31,2027.0,None,None,NaN,NaT,None,NaT,NaT,None,None,None,2021-03-01,22.0,-87111.0,NaT,No,None,None,None,None,RT - Rubber Tired Vintage Trolley,14.0,Yes,Owned outright by public agency (OOPA),None,TB - Trolleybus,None,Trolleybus,None,2023-10-13,2023-10-13
1138,Tehama County Transit Agency,1FDES6PG1LKB31822,1613144,978,NaN,Yes,Section 5311,PTMISEA,None,None,None,NaT,NaN,NaN,NaN,NaN,None,89655.00,74014.00,0.8255,15641.00,0.1744,NaN,NaN,NaN,NaN,NaN,None,Medium-Duty Bus,2020,Ford Motor Corporation,None,350 HD,GA - Gasoline,NaN,NaN,21.0,NaN,8.0,NaN,2.0,Lift,Braun,Yes,Active,None,5.0,2020-12-04,Excellent,111642.0,2023-08-31,2027.0,None,None,NaN,NaT,None,NaT,NaT,None,None,None,2020-12-03,4.0,88358.0,NaT,No,None,None,None,None,CU - Cutaway Bus,10.0,None,Leased or borrowed from related parties by a p...,None,CB - Commuter Bus,None,Demand Response,None,2023-10-13,2023-10-13
1139,Tehama County Transit Agency,1FDES6PG6LKB13816,163153,979,NaN,Yes,Section 5311,CA-18-X069 | 0015000075,None,None,None,NaT,NaN,NaN,NaN,NaN,None,92015.00,81498.00,0.8857,10517.00,0.1142,92015.0,1.0,NaN,NaN,NaN,None,Medium-Duty Bus,2020,Ford Motor Corporation,350HD,None,GA - Gasoline,NaN,NaN,21.0,NaN,10.0,NaN,NaN,Lift,Braun,None,Active,Replacement,5.0,2021-03-25,Excellent,39012.0,2023-08-31,NaN,None,None,NaN,NaT,None,NaT,NaT,None,None,None,2020-12-07,4.0,160988.0,NaT,No,None,None,None,None,CU - Cutaway Bus,10.0,None,None,None,None,None,None,None,2023-10-13,2023-10-13
1140,Tehama County Transit Agency,1FDES6PG8LKB31820,161

In [343]:
### ------- RR20F-182
# Total_Annual_Expenses_by_Mode for Capital expenses
newfleet_df = veh_df[veh_df['In_Service_Date'].dt.year == this_year ]
newfleet_df

,Organization,VIN,LicensePlate,AgencyId,DOTId,Funded_By_DOT,Funding_Program,Grant_Primary,Grant_Secondary,DOT_Contract,Purchase_Order_Number,Purchase_Order_Date,Invoice_Number,Payment_Voucher_Number,Payment_Voucher_Date,Award_Year,UPIN,Total_Cost,Federal_Share,Federal_Share_Percent,State_Share,State_Share_Percent,Local_Share,Local_Share_Percent,DOT_Capital_Responsibility,Organization_Capital_Responsibility,Other_Capital_Responsibility,Other_Capital_Responsibility_Org,Category,Vehicle_Year,Manufacturer,Model,Chassis,Fuel_Type,Vehicle_Height_Feet,Vehicle_Height_Inches,Vehicle_Length_Feet,GVWR,Seating_Capacity,Standing_Capacity,Wheelchair_Positions,Wheelchair_Ramp_Or_Lift,Wheelchair_Ramp_Or_Lift_Manufacturer,Emergency_Contingency_Vehicle,Status,Improvement_Purpose,Current_Condition,Current_Condition_Date,Condition_Rating,Current_Mileage,Current_Mileage_Date,Year_Of_Renewal,Renewal_Type,Price_Agreement_Number,_Price_Agreement_Date,Purchase_Date,Vendor,Received_Date,Last_Inspection_Date,Vehicle_Location_Street,Vehicle_Location_City,Vehicle_Location_State,In_Service_Date,Useful_Life_Years_Remaining,Useful_Life_Miles_Remaining,Out_Of_Service_Date,Is_DOT_LienHolder,Is_Title_On_File_With_DOT,Title_Number,Registration_Number,NTD_ID,Vehicle_Type,Average_Estimated_Service_Years_When_New,Dedicated_Fleet,Ownership_Type,Funding_Source,NTD_Other_Mode,NTD_Other_Type_Of_Service,Vehicle_Service_Modes_Comma_Delimited,Comments,date_uploaded_x,date_uploaded_y
1153,Tehama County Transit Agency,1FDVU4XG3NKA23994,CL00U92,981,NaN,Yes,None,None,None,None,None,NaT,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"Light-Duty Vans, Sedans or Buses",2023,Ford Motor Corporation,Ford,Ford E-350,GA - Gasoline,10.0,NaN,21.0,10360.0,10.0,0.0,2.0,Lift,Braun,Yes,Active,Replacement,5.0,2023-04-30,Excellent,6248.0,2023-08-31,NaN,None,None,NaN,NaT,A-Z Bus Sales,2023-01-26,2023-01-26,1509 Schwab Street,Red Bluff,CA,2023-01-31,4.0,93752.0,NaT,Yes,None,None,None,None,VN - Van,8.0,Yes,None,None,None,None,Demand Response,None,2023-10-13,2023-10-13
1159,Tehama County Transit Agency,5TDKSKFC8NS071225,1592360,METS 2,NaN,No,None,TBD,None,None,None,NaT,NaN,NaN,NaN,NaN,None,85000.0,0.0,0.0,0.0,0.0,85000.0,NaN,NaN,NaN,NaN,None,"Light-Duty Vans, Sedans or Buses",2022,Toyota Motor Corporation,Sienna,None,GA - Gasoline,6.0,4.0,10.0,6170.0,5.0,0.0,1.0,Ramp,Braun,Yes,Active,Expansion (New),5.0,2023-09-26,Excellent,4136.0,2023-08-31,NaN,None,None,NaN,NaT,None,2023-04-17,NaT,None,None,None,2023-06-01,3.0,95864.0,NaT,No,No,None,None,METS 2,MV - Minivan,8.0,Yes,None,None,None,None,Demand Response,None,2023-10-13,2023-10-13
1160,Tehama County Transit Agency,5TDKSKFCXNS71467,1592359,METS1,NaN,No,None,TBD,None,None,None,NaT,NaN,NaN,NaN,NaN,None,85000.0,0.0,0.0,0.0,0.0,85000.0,NaN,NaN,NaN,NaN,None,"Light-Duty Vans, Sedans or Buses",2022,Toyota Motor Corporation,Sienna,None,GA - Gasoline,6.0,4.0,10.0,6170.0,5.0,0.0,1.0,Ramp,Braun,Yes,Active,Expansion (New),5.0,2023-09-26,Excellent,5983.0,2023-08-31,NaN,None,None,NaN,NaT,None,2023-04-17,NaT,None,None,None,2023-06-01,3.0,94017.0,NaT,No,No,None,None,METS 1,MV - Minivan,8.0,Yes,None,None,None,None,Demand Response,None,2023-10-13,2023-10-13


In [354]:
any(pd.isna(newfleet_df['Ownership_Type']))

True

In [356]:
# try checking for year AND ownership type simultaneously
veh_df[(veh_df['In_Service_Date'].dt.year == this_year) & (veh_df['Ownership_Type'].str.contains("OOPA"))]

,Organization,VIN,LicensePlate,AgencyId,DOTId,Funded_By_DOT,Funding_Program,Grant_Primary,Grant_Secondary,DOT_Contract,Purchase_Order_Number,Purchase_Order_Date,Invoice_Number,Payment_Voucher_Number,Payment_Voucher_Date,Award_Year,UPIN,Total_Cost,Federal_Share,Federal_Share_Percent,State_Share,State_Share_Percent,Local_Share,Local_Share_Percent,DOT_Capital_Responsibility,Organization_Capital_Responsibility,Other_Capital_Responsibility,Other_Capital_Responsibility_Org,Category,Vehicle_Year,Manufacturer,Model,Chassis,Fuel_Type,Vehicle_Height_Feet,Vehicle_Height_Inches,Vehicle_Length_Feet,GVWR,Seating_Capacity,Standing_Capacity,Wheelchair_Positions,Wheelchair_Ramp_Or_Lift,Wheelchair_Ramp_Or_Lift_Manufacturer,Emergency_Contingency_Vehicle,Status,Improvement_Purpose,Current_Condition,Current_Condition_Date,Condition_Rating,Current_Mileage,Current_Mileage_Date,Year_Of_Renewal,Renewal_Type,Price_Agreement_Number,_Price_Agreement_Date,Purchase_Date,Vendor,Received_Date,Last_Inspection_Date,Vehicle_Location_Street,Vehicle_Location_City,Vehicle_Location_State,In_Service_Date,Useful_Life_Years_Remaining,Useful_Life_Miles_Remaining,Out_Of_Service_Date,Is_DOT_LienHolder,Is_Title_On_File_With_DOT,Title_Number,Registration_Number,NTD_ID,Vehicle_Type,Average_Estimated_Service_Years_When_New,Dedicated_Fleet,Ownership_Type,Funding_Source,NTD_Other_Mode,NTD_Other_Type_Of_Service,Vehicle_Service_Modes_Comma_Delimited,Comments,date_uploaded_x,date_uploaded_y


In [345]:
newfleet_df['AgencyId'].to_list()

['981', 'METS 2', 'METS1']

**Function and set up for function**

In [ ]:
# Get data for vehicle inventory
veh_inv_query = f"""SELECT * FROM `cal-itp-data-infra.blackcat_raw.{this_year}_revenue_vehicle_inventory`"""
veh_inv = client.query(veh_inv_query).to_dataframe()
orgs_query = f"""SELECT * FROM `cal-itp-data-infra.blackcat_raw.{this_year}_organizations`"""
orgs = client.query(orgs_query).to_dataframe()

veh_inv_orgs = (veh_inv.merge(orgs, on = 'Organization', 
                            indicator=True)
                .query('_merge == "both"').drop(columns=['_merge','date_uploaded_x', 'date_uploaded_y']))

In [400]:
def rr20f_182(inv_df, fin_df, year):
    agencies = fin_df['Organization_Legal_Name'].unique()
    output = []

    for agency in agencies:
        agency_df = fin_df[(fin_df['Organization_Legal_Name']==agency) & (fin_df['Fiscal_Year']==year)].drop_duplicates()
        agency_inv = inv_df[inv_df['Organization']==agency].drop_duplicates()
        
        if len(agency_df) == 0:
            print(f"There is no data for {agency} in {year}")
            continue
        
        if len(agency_inv) == 0:
            print(f"There is no inventory data for {agency} in {year}")
            continue
            
        total_cap_expenses = agency_df[agency_df['Operating_Capital']=='Capital']['Total_Annual_Expenses_by_Mode'].values[0]
        newfleet_df = agency_inv[(agency_inv['In_Service_Date'].dt.year == year) & (agency_inv['Ownership_Type'].str.contains("OOPA"))].drop_duplicates()

        if (len(newfleet_df) > 0) and (total_cap_expenses != 0):
            result = "pass"
            check_name = "RR20F-182: new fleet has capital expenses"
            description = ""
            print(f"{result} for {agency}")
        elif (len(newfleet_df) > 0) and (total_cap_expenses == 0):
            result = "fail"
            check_name = "RR20F-182: new fleet has capital expenses"
            description = f"There was $0 reported for Funds Expended on Capital for all modes on the RR-20 form, but {len(newfleet_df)} in the reporting year reported as Owned Outright by Public Agency (OOPA) in your inventory. Please provide narrative justification."
            print(f"{result} for {agency}")
        else:
            result = "warning"
            check_name = "RR20F-182: new fleet has capital expenses"
            description = f"Either capital expenses or inventory data is lacking. Check manually."
            print(f"{result} for {agency}")

        output_line = {"Organization": agency,
                   "name_of_check" : check_name,
                    "value_checked": f"New fleet OOPA={len(newfleet_df)}, Total_Annual_Expenses_by_Mode = ${total_cap_expenses}",
                    "check_status": result,
                    "Description": description}
        output.append(output_line)
    checks = pd.DataFrame(output).sort_values(by="Organization")
    return checks
            
            

In [401]:
pd.set_option('display.width', 1000)

numeric_columns = rr20_financial.select_dtypes(include=['number']).columns
rr20_financial[numeric_columns] = rr20_financial[numeric_columns].fillna(0)

rr20f_182(veh_inv_orgs, rr20_financial, this_year)

warning for County of Sacramento Department of Transportation
fail for Amador Transit
warning for San Benito County Local Transportation Authority
warning for City of Needles
warning for Alpine County Community Development
pass for Fresno County Rural Transit Agency
warning for Tuolumne County Transit Agency (TCTA)
warning for Tehama County Transit Agency
warning for City of Auburn
warning for City of Guadalupe
warning for Plumas County Transportation Commission
warning for County of Shasta Department of Public Works
warning for Madera County
warning for City of McFarland
warning for Mountain Area Regional Transit Authority
warning for Palo Verde Valley Transit Agency
warning for City of Corcoran - Corcoran Area Transit
warning for Eastern Sierra Transit Authority
warning for County of Nevada Public Works, Transit Services Division
warning for City of Solvang
warning for Redwood Coast Transit Authority
warning for City of Arvin
warning for Town of Truckee
warning for Morongo Basin Tran

,Organization,name_of_check,value_checked,check_status,Description
4,Alpine County Community Development,RR20F-182: new fleet has capital expenses,"New fleet OOPA=0, Total_Annual_Expenses_by_Mod...",warning,Either capital expenses or inventory data is l...
1,Amador Transit,RR20F-182: new fleet has capital expenses,"New fleet OOPA=2, Total_Annual_Expenses_by_Mod...",fail,There was $0 reported for Funds Expended on Ca...
21,City of Arvin,RR20F-182: new fleet has capital expenses,"New fleet OOPA=0, Total_Annual_Expenses_by_Mod...",warning,Either capital expenses or inventory data is l...
8,City of Auburn,RR20F-182: new fleet has capital expenses,"New fleet OOPA=0, Total_Annual_Expenses_by_Mod...",warning,Either capital expenses or inventory data is l...
30,City of California City,RR20F-182: new fleet has capital expenses,"New fleet OOPA=0, Total_Annual_Expenses_by_Mod...",warning,Either capital expenses or inventory data is l...
16,City of Corcoran - Corcoran Area Transit,RR20F-182: new fleet has capital expenses,"New fleet OOPA=0, Total_Annual_Expenses_by_Mod...",warning,Either capital expenses or inventory data is l...
31,City of Dixon,RR20F-182: new fleet has capital expenses,"New fleet OOPA=0, Total_Annual_Expenses_by_Mod...",warning,Either capital expenses or inventory data is l...
9,City of Guadalupe,RR20F-182: new fleet has capital expenses,"New fleet OOPA=0, Total_Annual_Expenses_by_Mod...",warning,Either capital expenses or inventory data is l...
13,City of McFarland,RR20F-182: new fleet has capital expenses,"New fleet OOPA=0, Total_Annual_Expenses_by_Mod...",warning,Either capital expenses or inventory data is l...
3,City of Needles,RR20F-182: new fleet has capital expenses,"New fleet OOPA=0, Total_Annual_Expenses_by_Mod...",warning,Either capital expenses or inventory data is l...
